In [ ]:
!pip install librosa

In [ ]:
!pip install moviepy

In [ ]:
!pip install pyo

In [3]:

import time, random

In [11]:

# from moviepy.editor import VideoFileClip, concatenate_videoclips


In [8]:

import numpy as np
import librosa
# from moviepy.editor import *
from moviepy import VideoFileClip, AudioFileClip, concatenate_videoclips

In [ ]:
!python -m pip install wxPython

In [10]:
!pip install regex

In [13]:
import pyo
from pyo import *

In [14]:

import re, unicodedata, math
import numpy as np
from collections import Counter


In [15]:

# Download Unicode Scripts.txt (nned to run only once; you can also cache this locally)
SCRIPTS_URL = "https://www.unicode.org/Public/UCD/latest/ucd/Scripts.txt"


# ALL SCRIPTS 'UNDER THE SUN' [IN UNICODE, THAT IS]

import regex
import urllib.request


def get_all_scripts() -> set[str]:
    """
    Fetch the official Unicode script names from Scripts.txt.
    """
    with urllib.request.urlopen(SCRIPTS_URL) as f:
        lines = f.read().decode("utf-8").splitlines()

    scripts = set()
    for line in lines:
        if line.strip() and not line.startswith("#"):
            # Example line: "0041..005A; Latin # L&  [26] LATIN CAPITAL LETTER A..Z"
            parts = line.split(";")
            if len(parts) >= 2:
                script = parts[1].strip().split()[0]
                scripts.add(script)
    return scripts

UNICODE_SCRIPTS = sorted(get_all_scripts())

def char_script(ch):
    import regex
    
    if not ch or len(ch) != 1:
        return "INVALID"

    for script in UNICODE_SCRIPTS:
        try:
            # Use the script name exactly as Unicode defines it
            if regex.match(rf"\p{{Script={script}}}", ch):
                return script  # return it as-is
        except regex.error:
            continue  # skip invalid/unrecognized scripts

    return "UNKNOWN"


from collections import Counter

def word_script(word: str) -> str:
    """
    Return the dominant script of a word (based on majority of alphabetic chars).
    """
    scripts = Counter(char_script(ch) for ch in word if ch.isalpha())
    return scripts.most_common(1)[0][0] if scripts else "OTHER"


def get_unicode_name(ch):
    try:
        return unicodedata.name(ch)
    except ValueError:
        return None


import unicodedata

# Latin/Cyrillic/Greek/Devanagari vowels (extendable)
_vowel_re_latin = re.compile(r"[aeiouy\u00E0-\u00FF]+", re.IGNORECASE)
_vowel_re_cyrillic = re.compile(r"[аеёиоуыэюя]+", re.IGNORECASE)  # basic Russian vowels
_vowel_re_greek = re.compile(r"[αεηιουωάέήίόύώ]", re.IGNORECASE)   # modern Greek vowels
_vowel_re_devanagari = re.compile(r"[अआइईउऊएऐओऔऋॠॡॢॣ]", re.IGNORECASE)


In [ ]:
!pip install nltk

In [17]:

def approx_syllables_word(word: str) -> int:
    if not word:
        return 0
    w = unicodedata.normalize("NFC", word)
    script = word_script(w)

    if script == "LATIN":
        groups = _vowel_re_latin.findall(w)
        count = len(groups)
        if w.lower().endswith("e") and count > 1:  # silent 'e'
            count -= 1
        return max(1, count)

    if script == "CYRILLIC":
        groups = _vowel_re_cyrillic.findall(w)
        return max(1, len(groups))

    if script == "GREEK":
        groups = _vowel_re_greek.findall(w)
        return max(1, len(groups))

    if script == "DEVANAGARI":
        groups = _vowel_re_devanagari.findall(w)
        return max(1, len(groups))

    if script in ("HIRAGANA", "KATAKANA"):
        kana_chars = [ch for ch in w if '\u3040' <= ch <= '\u30FF']
        return max(1, len(kana_chars))

    if script == "HANGUL":
        return len([ch for ch in w if '\uAC00' <= ch <= '\uD7A3'])

    if script == "CJK":
        chars = [ch for ch in w if '\u4E00' <= ch <= '\u9FFF']
        return max(1, len(chars))

    if script == "THAI":
        return max(1, len([ch for ch in w if ch.strip()]))

    # Fallback
    groups = _vowel_re_latin.findall(w)
    return max(1, len(groups) if groups else len(w))


def extract_phonological_clusters(word: str):
    clusters = set()
    w = unicodedata.normalize("NFC", word.lower())
    script = word_script(w)

    if script in ("LATIN", "GREEK", "CYRILLIC"):
        consonant_matches = re.findall(r'[^aeiouy]+', w)
        for c in consonant_matches:
            for i in range(len(c)):
                for j in range(i+1, len(c)+1):
                    clusters.add(c[i:j])
        vowel_matches = re.findall(r'[aeiouy]+', w)
        for v in vowel_matches:
            for i in range(len(v)):
                for j in range(i+1, len(v)+1):
                    clusters.add(v[i:j])
        for k in range(2, 5):
            if len(w) >= k:
                clusters.add(w[-k:])

    elif script in ("ARABIC", "HEBREW"):
        consonant_runs = re.findall(r'[^aeiou]+', w)
        for c in consonant_runs:
            for i in range(len(c)):
                for j in range(i+1, len(c)+1):
                    clusters.add(c[i:j])
        for k in range(2, 5):
            if len(w) >= k:
                clusters.add(w[-k:])

    elif script == "DEVANAGARI":
        groups = _vowel_re_devanagari.findall(w)
        for g in groups:
            clusters.add(g)
        for k in range(2, 5):
            if len(w) >= k:
                clusters.add(w[-k:])

    elif script in ("HIRAGANA", "KATAKANA", "HANGUL", "CJK"):
        chars = list(w)
        clusters.update(chars)
        for i in range(len(chars)-1):
            clusters.add(chars[i] + chars[i+1])

    else:
        for i in range(len(w)):
            for j in range(i+1, min(i+4, len(w))+1):
                clusters.add(w[i:j])

    return clusters


_word_re = re.compile(r"\w+", re.UNICODE)

def tokenize_text(text):
    tokens = []
    for m in _word_re.finditer(text):
        tok = m.group(0)
        tokens.append(tok)
    return tokens

_fricatives = set(list("fvsz") + ["sh","zh","th"])
_plosives = set(list("pbtdkg"))

def phonetic_density(tokens):
    latin_tokens = [t for t in tokens if char_script(t[0]) == "LATIN"]
    joined = " ".join(latin_tokens).lower()
    letters = re.sub(r'[^a-z]', '', joined)
    if not letters:
        return 0.0, 0.0, 0.0
    fric_count = sum(joined.count(f) for f in ["f","v","s","z","sh","zh","th"])
    plos_count = sum(joined.count(p) for p in ["p","b","t","d","k","g"])
    vowel_count = sum(1 for c in letters if c in "aeiouy")
    total = len(letters)
    return fric_count/total, plos_count/total, vowel_count/(total+1e-9)

import re
import nltk
from nltk.corpus import words, stopwords, wordnet as wn
from nltk.corpus.reader import WordListCorpusReader
import numpy as np

# Download necessary NLTK resources
nltk.download('words')
nltk.download('stopwords')
nltk.download('omw')
nltk.download('omw-1.4')

# Load English words and stopwords
english_words = set(words.words())
stop_words = set(stopwords.words('english'))

# Define common prefixes and suffixes
PL_PREFIXES = {"re", "un", "in", "dis", "pre", "sub"}
PL_SUFFIXES = {"ing", "ed", "er", "ly", "es", "ful"}

def is_plausible_fragment(fragment):
    """Check if fragment is a plausible English word, prefix/suffix, or foreign fragment."""
    fragment = fragment.lower()
    if not fragment:
        return False
    if fragment in english_words:
        return True
    if fragment in PL_PREFIXES or fragment in PL_SUFFIXES:
        return True
    # Check if fragment exists in WordNet for any language
    for lang in wn.langs():
        if wn.synsets(fragment, lang=lang):
            return True
    # Fallback: accept fragments that are at least 2 characters long
    if len(fragment) > 1:
        return True
    return False

def extract_audio_features_from_stanza(stanza, expected_feet_per_line=(5,6), foot_syllables=(2,3)):
    lines = [ln.strip() for ln in stanza.strip().split("\n") if ln.strip()]
    n_lines = max(1, len(lines))
    tokens = tokenize_text(stanza)
    syll_counts_tokens = [approx_syllables_word(t) for t in tokens]
    total_syllables = sum(syll_counts_tokens)
    n_words = len(tokens) if tokens else 1
    syllable_density = total_syllables / n_words if n_words else 0.0
    target_feet = np.mean(expected_feet_per_line)
    avg_syll_per_line = total_syllables / max(1, len(lines))
    avg_foot_syll = np.mean(foot_syllables)
    tempo = avg_syll_per_line / avg_foot_syll
    sylls_per_line = [sum(approx_syllables_word(t) for t in tokenize_text(ln)) for ln in lines]
    pacing_variance = float(np.var(sylls_per_line)) if sylls_per_line else 0.0
    fric_density, plos_density, vowel_ratio = phonetic_density(tokens)
    vocal_smoothness = float(vowel_ratio)

    # --- [word-splitting] enjambment detection (needed in this specific case; if you need to process enjambments in general see https://github.com/Margento/Computationally_Assembled_Belgian_Poetry_Anthology ---
    enjambments = 0
    enjambed_positions = set()

    for ln_idx, ln in enumerate(lines):
        # 1. End-of-line split (including ellipses)
        end_match = re.search(r'(\w+(?:\.\.\.)?)-/?(\w*)$', ln)
        if end_match:
            left, right = end_match.groups()
            if is_plausible_fragment(left) and (not right or is_plausible_fragment(right)):
                enjambments += 1
                enjambed_positions.add(end_match.start())

        # 2. Start-of-line split
        if ln_idx > 0:
            start_match = re.match(r'^(\w*)-/(\w+)', ln)
            if start_match:
                left, right = start_match.groups()
                if (not left or is_plausible_fragment(left)) and is_plausible_fragment(right):
                    enjambments += 1
                    enjambed_positions.add(start_match.start())

        # 3. Multi-word or foreign-word consideration (fallback)
        for match in re.finditer(r'(\S+)/(\S+)', ln):
            left, right = match.groups()
            if is_plausible_fragment(left) and is_plausible_fragment(right):
                enjambments += 1
                enjambed_positions.add(match.start())

    # Count pause marks excluding those part of valid enjambments
    pause_marks = 0
    for m in re.finditer(r'[,;:\-\—\(\)]', stanza):
        if m.start() not in enjambed_positions:
            pause_marks += 1

    silence_ratio = pause_marks / (total_syllables + 1e-9)
    caesura = sum(1 for ln in lines if "," in ln or ";" in ln or "—" in ln)

    enjambments_norm = enjambments / n_lines
    caesura_norm = caesura / n_lines

    audio = {
        "syllable_density": float(syllable_density),
        "tempo": float(tempo),
        "pacing_variance": float(pacing_variance),
        "fricative_density": float(fric_density),
        "plosive_density": float(plos_density),
        "vocal_smoothness": float(vocal_smoothness),
        "silence_ratio": float(silence_ratio),
        "total_syllables": int(total_syllables),
        "sylls_per_line": sylls_per_line,
        "enjambments": float(enjambments_norm),
        "caesura": float(caesura_norm),
        "n_words": n_words
    }
    return audio


[nltk_data] Downloading package words to /Users/raluca/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raluca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to /Users/raluca/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/raluca/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
!pip install torch torchvision torchaudio

In [19]:

# IN CASE YOU ARE ON MPS

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple Silicon GPU via MPS")
else:
    device = torch.device("cpu")
    print("⚠️ MPS not available, falling back to CPU")


✅ Using Apple Silicon GPU via MPS


In [ ]:
!pip install tiktoken

In [ ]:
!pip install --upgrade protobuf sentencepiece

In [22]:
import math

In [ ]:
!pip install transformers

In [19]:

from transformers import XLMRobertaTokenizer, AutoModelForSequenceClassification, pipeline
# import torch

# Model + tokenizer
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)  # force slow tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device
)


Device set to use mps


In [28]:

# TESTING THE AFFECT PIPELINE
print(sentiment_pipeline("This is working now! 🚀"))
print(sentiment_pipeline("I am loving this! ❤️"))

[{'label': 'positive', 'score': 0.7629498839378357}]
[{'label': 'positive', 'score': 0.944301426410675}]


In [24]:

# AFFECT/SENTIMENT ANALYSIS

import math

def stanza_affect_vector(stanza):
    """
    Extract affective features (valence, arousal, energy) from a stanza of text,
    combining multilingual sentiment analysis with audio-like features.
    """

    # --- 1. Multilingual Sentiment Analysis (Hugging Face) ---
    try:
        sentiment_result = sentiment_pipeline(stanza[:512])[0]  # truncate to model limit
        label = sentiment_result["label"].lower()
        score = sentiment_result["score"]

        # Map labels to a polarity value in [-1, 1]
        if "negative" in label:
            polarity = -score
        elif "positive" in label:
            polarity = score
        else:  # neutral
            polarity = 0.0
    except Exception as e:
        print(f"Sentiment analysis failed: {e}")
        polarity = 0.0

    # Calculate valence from polarity
    valence = float(math.tanh(polarity * 5.0))

    # --- 2. Extract "audio" features ---
    audio_feats = extract_audio_features_from_stanza(stanza)

    # --- 3. Calculate arousal & energy ---
    arousal = (audio_feats["pacing_variance"] ** 0.5
               + audio_feats["fricative_density"] * 0.5
               + min(1.0, audio_feats["silence_ratio"] * 2.0))
    arousal = float(math.tanh(arousal))

    energy = float(math.tanh(
        (audio_feats["tempo"] * 0.6) +
        (audio_feats["syllable_density"] * 0.2)
    ))

    return {
        "valence": valence,
        "arousal": arousal,
        "energy": energy,
        # "audio_feats": audio_feats
    }

In [25]:

# TEMPORAL FEATURES
def stanza_temporal_structures(stanza):
    lines = [ln.strip() for ln in stanza.strip().split("\n") if ln.strip()]
    n_segments = len(lines)
    segment_annotations = []
    motifs_counter = Counter()
    
    # process clusters instead of whole words
    for i, ln in enumerate(lines):
        words = tokenize_text(ln)
        segment_annotations.append(
            f"line_{i+1}: {len(words)} words, {sum(approx_syllables_word(w) for w in words)} sylls"
        )
        for w in words:
            clusters = extract_phonological_clusters(w)
            for c in clusters:
                motifs_counter[c] += 1
    
    motifs = [cl for cl,cnt in motifs_counter.items() if cnt > 1]
    n_motifs = len(motifs)
    n_uniques = len(motifs_counter)
    
    sylls_per_line = [
        sum(approx_syllables_word(w) for w in tokenize_text(ln)) for ln in lines
    ] if lines else []
    
    ruptures = []
    if sylls_per_line:
        mean = np.mean(sylls_per_line); sd = np.std(sylls_per_line)
        for i, s in enumerate(sylls_per_line):
            if sd > 0 and abs(s-mean) > 1.5*sd:
                ruptures.append({
                    "line": i+1, 
                    "syllables": int(s), 
                    "deviation": float((s-mean)/sd)
                })
    
    score_linear = 0.0
    if len(sylls_per_line) > 1:
        x = np.arange(len(sylls_per_line))
        y = np.array(sylls_per_line)
        cov = np.cov(x, y)[0,1]
        if np.std(x) > 0 and np.std(y) > 0:
            score_linear = float(cov / (np.std(x) * np.std(y)))
    
    score_cyclical = 0.0
    if len(sylls_per_line) > 2:
        y = np.array(sylls_per_line) - np.mean(sylls_per_line)
        score_cyclical = float(np.correlate(y, np.roll(y,1))[0] / (np.sum(y*y)+1e-9))
    
    ngrams = Counter()
    for ln in lines:
        toks = [t.lower() for t in tokenize_text(ln)]
        for i in range(len(toks)-1):
            ngrams[" ".join(toks[i:i+2])] += 1
    repeated_ngrams = sum(1 for c in ngrams.values() if c>1)
    score_recursive = float(repeated_ngrams / (len(ngrams)+1e-9))
    score_hybrid = float((abs(score_linear) + abs(score_cyclical) + score_recursive)/3.0)
    
    recursive_events = motifs[:5]
    
    return {
        "segments": n_segments,
        "segment_annotations": segment_annotations,
        "number_of_motifs": n_motifs,
        "motifs": motifs,
        "uniques": n_uniques,
        "ruptures": ruptures,
        "score_linear": round(score_linear, 3),
        "score_cyclical": round(score_cyclical, 3),
        "score_recursive": round(score_recursive, 3),
        "score_hybrid": round(score_hybrid, 3),
        "recursive_events": recursive_events
    }

def extract_full_stanza_representation(stanza):
    audio_feats =  extract_audio_features_from_stanza(stanza)
    affect = stanza_affect_vector(stanza)
    temporal = stanza_temporal_structures(stanza)

    return {
        "audio_features": audio_feats,
        "affect_vector": affect,
        "temporal_features": temporal,
    }

In [26]:

# TEXT OF "World Poem I. Foundational Quantum Mechanics, AHiRA, & CMoLA" by MARGENTO
# REPLACE WITH YOUR OWN STANZAS

stanza_0 = """
         W   que			   S  outh
   in		  quirir	         to	      	            chase
       con		quirimos	the	      	chases
	      ad			             per
"""

stanza_1 = """
uje adujo adujiste adari adz adâncu dus d(vr)ut
	   duce ado thaws injudicious, odd airy $nazzy Duke’s di(¢)tion (Φ)root 

   	Ocaso que no hemos todavía podido averisnar quién es el auto
   	   acasă un sun$eth to dive in, airpods on, getting aut(h)o® ads

	La obra musical, como el animal, transmite la vida que ha recibido
	   for being musical is being animal, trans smithing avidly the recipe tome
"""

stanza_2 = """
No 	
        pue	 	son
           den 		         
     ser 	                  sí
      		   hijos       
        del             legítimos 
              	 Arti		 Arte
        		fici
o
			
			f
			Φi	 $(i)on
			¢€	   ¢$ 
		      re
		           ®
	           Aut(E)o
	                     outer
                                 oughta
                            ghτer  
the           Δau
"""

stanza_3 = """
         Y el afán de identificarse con tualidad; porque en $uma, 
being a fan for trendy farces by con dads, tu-i porcu’i muma

         ¿qué son los poetas más – ella no lleva a la muerte, 
de sonorous poetic mess—mass, Ella & Billie style assertive

         sino a una vida – que los poseedores del maravilloso
sinning ’n g’ luna void key low, posey odors, a deli marries via Zoso …

         poder expreso – más plena y más completa… 
power ’v eXpress oh--my ass, plain, I must complain …
"""

stanza_4 = """
soñ	EN señ          son	        E$ sign
  que           ab[β]a 	    ic	       lava
		ni		               t @t €
       ro	           un	     gr	         ion
	    lib			  εr®
"""

stanza_5 = """
                                                      en secreto
El hombre que conSt mi sueño, los ojos, el fardo, nos dicen – amor, su alfabeto –
                                    minä’ tejin’ käminä’		        ὕ   
   all amber, Zoque it, mine me sayin’, insecure sod — a morsel, all, the fabri¢ back-to

                                						    posesión de
profundizar ES materias, nuidad de la existencia, entra o toma un organismo físico,
         τ ℕ	         $					                 Poseidon’s twin
   professionals ’n’ materials, nudity delete extinction, enter ol’ Thomas and comes mystically

cn que se trata	   en la mente d
en nOS tribunales, penetrar    el médium y darle expresión % escritura automática
   as the letter c                 a lamented d			        Zong!	     ray  awe		  τ
   in matriarchal menace, prota amari,       e-al£ made in his im®age, ex carbON ton(€)omous Arcti¢a

"""

stanza_6 = """
           es una
   e££a 		  his
  nov		   tor		             que
             	              i			  Φ is i   x
        anu se     a		           	                $
		    ®t		               no
			i  	Φ ic i  al  $α
da		dă		    wa   a do
  vi	vi
		     £
pAso	a PaSo	
		$tep	by    s(To)p   litter
			    be	   AM
"""

In [22]:

features_0 = extract_full_stanza_representation(stanza_0)
features_1 = extract_full_stanza_representation(stanza_1)
features_2 = extract_full_stanza_representation(stanza_2)
features_3 = extract_full_stanza_representation(stanza_3)
features_4 = extract_full_stanza_representation(stanza_4)
features_5 = extract_full_stanza_representation(stanza_5)
features_6 = extract_full_stanza_representation(stanza_6)

In [23]:

#stanzas = []
stanzas = [features_0, features_1, features_2, features_3, features_4, features_5, features_6]

In [ ]:

!pip install pydub ffmpeg-python


In [ ]:

# AUDIO


In [39]:
stanza_lengths = [15, 25, 25, 35, 20, 75, 25]

In [28]:

"""
Sonification engine (pyo)
Adds optional layers:
 - self-referential recursion feedback (Option 3)
 - lexical uniqueness shimmer
 - recursive-motif pings
"""


# ---------------- CONFIG ----------------
use_recursion_feedback = True      # stanza-level self-reference
use_uniques_layer      = True      # lexical diversity shimmer
use_recursive_pings    = True      # motif memory pings

fade_time = 2.0
# stanza_lengths = [15, 25, 25, 35, 20, 75, 25] # REPLACE WITH DURATIONS OF YOUR OWN STANZAS

# ---------------- MAPPINGS ----------------
def tempo_to_freq(tempo, base_freq=110.0, scale=2.0, tempo_ref=10.0,
                  min_freq=55.0, max_freq=1760.0):
    tempo = max(0.0001, float(tempo))
    freq = base_freq * (scale ** (math.log(tempo / tempo_ref, 2)))
    return max(min_freq, min(max_freq, freq))

def energy_to_amp(e): return max(0.01, min(1.2, float(e))) * 0.6
def arousal_to_bright(a): return max(0.0, min(1.0, float(a)))
def fric_to_noise(f): return max(0.0, min(1.0, float(f)))
def plosive_to_click_strength(p): return max(0.0, min(1.5, float(p)))
def vocal_smooth_to_cutoff(vs): return 500.0 + float(vs) * 5500.0

# ---------------- SERVER ----------------
s = Server(duplex=0).boot()
s.start()
# master_mix = Sig(0)
# rec_master = Record(master_mix, filename="world_poem_1_sonification_master.wav",
                    # fileformat=0, sampletype=1)

# rec_master = Record(s.amp, filename="world_poem_1_sonification_master.wav",
                    # fileformat=0, sampletype=1)

# rec_master.start()

# ---------------- HELPERS ----------------
def make_rupture_schedule(stanza_idx, stanza, start, dur):
    out, tf = [], stanza.get("temporal_features", {})
    segs = tf.get("segments", 0)
    for r in tf.get("ruptures", []):
        frac = float(r.get("line", 1)-1) / max(1, segs)
        strength = min(1.6, abs(r.get("deviation", 0)))
        out.append((start + frac*dur, strength))
    return out

def trigger_rupture_burst(strength, panpos):
    n = Noise()
    hf = Sine(freq=1000+2000*strength)
    env = Adsr(0.002, 0.08*strength, 0, 0.02, dur=0.15).play()
    nenv = Adsr(0.001, 0.05*strength, 0, 0.01, dur=0.12).play()
    Pan(hf*0.6*strength*env + n*0.3*strength*nenv, pan=panpos).out()

# ---------------- MAIN ----------------

import os, datetime

folder_name = f"world_poem_1_stanza_wavs_{datetime.date.today().isoformat()}"
os.makedirs(folder_name, exist_ok=True)
print(f"🎧 Stanza recordings will be saved to: {folder_name}/")


global_time = 0.0
for i, stanza in enumerate(stanzas):
    dur = stanza_lengths[i]
    af, av, tf = stanza["audio_features"], stanza["affect_vector"], stanza["temporal_features"]

    base_freq = tempo_to_freq(af.get("tempo", 4.0))
    base_amp  = energy_to_amp(av.get("energy", 0.8))
    bright    = arousal_to_bright(av.get("arousal", 0.5))
    noise_lvl = fric_to_noise(af.get("fricative_density", 0.0))
    plosive_strength = plosive_to_click_strength(af.get("plosive_density", 0.0))
    cutoff    = vocal_smooth_to_cutoff(af.get("vocal_smoothness", 0.3))
    score_lin = tf.get("score_linear", 0.0)
    score_cyc = tf.get("score_cyclical", 0.0)
    score_rec = tf.get("score_recursive", 0.0)

    print(f"▶ Stanza {i+1} freq≈{base_freq:.1f}Hz amp≈{base_amp:.2f}")

    amp_ctrl = SigTo(value=0.0, time=fade_time, init=0.0)

    # pacing variance “breathing”
    pv = float(af.get("pacing_variance", 1.0))
    lfo_rate  = 0.05 + min(0.8, pv / 25.0)
    lfo_depth = 2.0 + min(20.0, pv * 0.6)
    pacing_lfo = Sine(freq=lfo_rate, mul=lfo_depth)

    # wander
    wander = SigTo(base_freq, time=5.0)
    def new_wander(): wander.value = base_freq * (1.0 + random.uniform(-0.02,0.02))
    Pattern(new_wander, time=max(2.0, dur/6.0)).play()

    # optional recursion feedback (metamorphic)
    fb = Sig(0)
    if use_recursion_feedback:
        rec_gain = min(0.25, score_rec * 0.5)
        fb_lfo = Sine(freq=0.1 + score_rec * 0.3, mul=base_freq * rec_gain)
        modulated_freq = base_freq + pacing_lfo + wander + fb_lfo + fb
    else:
        modulated_freq = base_freq + pacing_lfo + wander

    vib_lfo = Sine(freq=0.15, mul=3.0)
    carrier = Sine(freq=modulated_freq + vib_lfo, mul=0.0)
    if use_recursion_feedback: fb.value = carrier * rec_gain

    carrier_amp = carrier * amp_ctrl
    overtone = Sine(freq=modulated_freq*1.98, mul=bright*0.2)
    noise    = Noise(mul=noise_lvl*0.3)
    tonal    = ButLP(carrier_amp + overtone + noise, freq=cutoff)

    # stereo motion
    pan_lfo  = Sine(freq=0.02 + abs(score_cyc)*0.02, mul=0.25, add=0.5)
    lin_drift = SigTo(float(score_lin)*0.2, time=dur)
    pan_sig  = Clip(pan_lfo + lin_drift, 0.0, 1.0)
    stereo   = Pan(tonal, pan=pan_sig).out()

    amp_ctrl.value = base_amp
    # rec_stanza = Record(stereo, filename=f"stanza_{i+1}.wav",
                        # fileformat=0, sampletype=1)
    rec_stanza = Record(
    stereo,
    filename=os.path.join(folder_name, f"stanza_{i+1}.wav"),
    fileformat=0,
    sampletype=1
)
    # rec_stanza.start()

    # ruptures
    for rt, strength in make_rupture_schedule(i, stanza, global_time, dur):
        CallAfter(lambda s=strength: trigger_rupture_burst(s, random.uniform(0.3,0.7)),
                  rt - global_time)

    # plosive clicks
    if plosive_strength > 0.05:
        m = Metro(time=1.0/max(0.2, plosive_strength*2.0)).play()
        env = TrigEnv(m, table=HannTable(), dur=0.06, mul=plosive_strength*0.08)
        Pan(Sine(freq=base_freq*1.5, mul=env), pan=random.uniform(0.3,0.7)).out()

    # uniqueness shimmer
    if use_uniques_layer:
        uniques = tf.get("uniques", 0)
        uniq_density = min(0.8, 0.1 + uniques / 750.0)
        for _ in range(int(5 + uniq_density*20)):
            Sine(freq=random.uniform(1000,6000),
                 mul=uniq_density*0.02).out()

    # recursive motif pings
    if use_recursive_pings:
        rec_events = tf.get("recursive_events", [])
        ping_rate = max(0.3, 8.0/(len(rec_events)+2))
        met = Metro(time=ping_rate).play()
        env = TrigEnv(met, HannTable(), dur=0.08, mul=0.04)
        Pan(Sine(freq=660+len(rec_events)*3, mul=env),
            pan=random.random()).out()

    time.sleep(dur)
    amp_ctrl.value = 0.0
    time.sleep(fade_time)
    rec_stanza.stop()
    global_time += dur

# ---------------- SHUTDOWN ----------------
time.sleep(0.5)
rec_master.stop()
# print("✅ Master recording finished: world_poem_1_sonification_master.wav")
print("✅ Recording finished for all stanzas")
s.stop(); s.shutdown()


Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.
🎧 Stanza recordings will be saved to: world_poem_1_stanza_wavs_2025-10-08/
▶ Stanza 1 freq≈55.0Hz amp≈0.53
▶ Stanza 2 freq≈93.9Hz amp≈0.60
▶ Stanza 3 freq≈55.0Hz amp≈0.37
▶ Stanza 4 freq≈75.3Hz amp≈0.60
▶ Stanza 5 freq≈55.0Hz amp≈0.53
▶ Stanza 6 freq≈102.4Hz amp≈0.60
▶ Stanza 7 freq≈55.0Hz amp≈0.48
✅ Recording finished for all stanzas


In [29]:

from pydub import AudioSegment
import glob, os

# Directory containing stanza WAVs
input_dir = "world_poem_1_stanza_wavs_2025-10-08"
output_wav = "world_poem_1_full_mix.wav"

# Find all stanza files and sort numerically
files = sorted(glob.glob(os.path.join(input_dir, "stanza_*.wav")),
               key=lambda x: int("".join(filter(str.isdigit, os.path.basename(x))) or 0))

if not files:
    raise FileNotFoundError("No stanza_*.wav files found!")

print(f"Found {len(files)} stanza files. Concatenating...")

full_mix = AudioSegment.silent(duration=0)

for f in files:
    seg = AudioSegment.from_wav(f)
    full_mix += seg
    print(f" → Added {os.path.basename(f)} ({len(seg)/1000:.1f}s)")

# Export combined WAV
full_mix.export(output_wav, format="wav")
print(f"\n✅ Exported full mix to {output_wav}")


Found 7 stanza files. Concatenating...
 → Added stanza_1.wav (17.0s)
 → Added stanza_2.wav (27.0s)
 → Added stanza_3.wav (27.0s)
 → Added stanza_4.wav (37.0s)
 → Added stanza_5.wav (22.0s)
 → Added stanza_6.wav (77.0s)
 → Added stanza_7.wav (27.0s)

✅ Exported full mix to world_poem_1_full_mix.wav


In [ ]:

# VIDEO


In [31]:
import os, json, math, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from PIL import Image, ImageDraw, ImageFont
#from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips, AudioFileClip
from moviepy import VideoClip, AudioFileClip, concatenate_videoclips
import librosa
from scipy.io import wavfile
from scipy.spatial.distance import cosine
import difflib

In [ ]:
!pip install matplotlib scipy pillow sentence-transformers pyphonetics jellyfish

In [ ]:

# semantic embeddings
# if not available, fallback option, difflib
try:
    from sentence_transformers import SentenceTransformer
    SENT_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
    print("Loading sentence-transformer", SENT_MODEL)
    s_model = SentenceTransformer(SENT_MODEL)
    def semantic_similarity(a,b):
        if not a or not b: return 0.0
        ea = s_model.encode(a, convert_to_numpy=True)
        eb = s_model.encode(b, convert_to_numpy=True)
        sim = float(np.dot(ea, eb) / (np.linalg.norm(ea)*np.linalg.norm(eb) + 1e-12))
        return max(0.0, min(1.0, (sim+1)/2))
except Exception:
    s_model = None
    import difflib
    def semantic_similarity(a,b):
        if not a or not b: return 0.0
        return difflib.SequenceMatcher(None, a, b).ratio()

In [95]:

# Final integrated version: labels fade-in, pulse couplet flashes, glyph overlays, dynamic plots, recap.


# phonetics
try:
    from pyphonetics import Metaphone
    metaphone = Metaphone()
    def phonetic_code(s): return metaphone.phonetics(s or "")
except Exception:
    import jellyfish
    def phonetic_code(s): return jellyfish.metaphone(s or "")


JSON_PATH = "world_poem_1_structure.json" # REPLACE THIS WITH YOUR OWN POEM (STRUCTURE) JSON
AUDIO_MASTER = "world_poem_1_full_mix.wav"

TMP_DIR = Path("tmp_frames_v2"); TMP_DIR.mkdir(exist_ok=True)
W, H = 1280, 720
FPS = 24
FINAL_LABEL_SECONDS = 3.0
ORBIT_LABEL_FADE_STEPS = 10
COUPLET_FLASH_SECONDS = 0.8   # each ES or EN flash
GLYPH_FLASH_SECONDS = 0.6
MIN_COUPLET_SECONDS = 1.0
# FONT_PATH = None
FONT_PATH = "Arial_Unicode.ttf" # If you don't have this (or any other font), comment out and uncomment the line above
FONT = None


def get_font(sz):
    global FONT
    if FONT is not None: return FONT
    try:
        if FONT_PATH:
            FONT = ImageFont.truetype(FONT_PATH, sz)
        else:
            FONT = ImageFont.load_default()
    except Exception:
        FONT = ImageFont.load_default()
    return FONT


def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_pil_img(img, p):
    p.parent.mkdir(parents=True, exist_ok=True)
    img.convert("RGB").save(p)

def pil_to_np(img):
    return np.array(img.convert("RGB"))

def make_blank_frame(color=(6,6,10)):
    return Image.new("RGB", (W,H), color)

def paste_center(bg, fg, offset=(0,0)):
    bx, by = bg.size
    fx, fy = fg.size
    x = (bx - fx)//2 + offset[0]
    y = (by - fy)//2 + offset[1]
    b2 = bg.copy()
    b2.paste(fg, (x,y), fg if fg.mode=="RGBA" else None)
    return b2

def phonetic_similarity(a,b):
    ca = phonetic_code(a or "")
    cb = phonetic_code(b or "")
    if not ca and not cb: return 0.0
    import difflib
    return float(difflib.SequenceMatcher(None, ca, cb).ratio())


def build_orbit_graph_from_stanza(stanza):
    units=[]
    for li, line in enumerate(stanza.get("lines",[])):
        for ord_idx, u in enumerate(line.get("orbit_units",[])):
            uid = u.get("id", f"l{li}_u{ord_idx}_{u.get('text')}")
            units.append({"id": uid, "text": u.get("text"), "line": li, "ord": ord_idx, "raw":u})
    id_map = {u["id"]:u for u in units}
    def match_by_text(target):
        res=[]
        for u in units:
            if target.lower() in (u["text"] or "").lower():
                res.append(u["id"])
        return res
    edges={}
    def add_edge(a,b,w):
        if a==b: return
        key = tuple(sorted((a,b)))
        edges[key] = max(edges.get(key,0.0), float(w))
    # adjacency and same-line non-neighbors
    for u in units:
        for v in units:
            if u["line"]==v["line"]:
                d = abs(u["ord"]-v["ord"])
                if d==1:
                    add_edge(u["id"], v["id"], 1.0)
                elif d>1:
                    add_edge(u["id"], v["id"], 0.5)
    # combine_with
    for u in units:
        comb = u["raw"].get("combine_with",[]) or []
        for target in comb:
            if target in id_map:
                v = id_map[target]
                if v["line"]==u["line"] and abs(v["ord"]-u["ord"])==1:
                    add_edge(u["id"], v["id"], 1.5)
                else:
                    add_edge(u["id"], v["id"], 2.0)
            else:
                matches = match_by_text(target)
                if matches:
                    for mid in matches:
                        v = id_map[mid]
                        if v["line"]==u["line"] and abs(v["ord"]-u["ord"])==1:
                            add_edge(u["id"], mid, 1.5)
                        else:
                            add_edge(u["id"], mid, 2.0)
                else:
                    tid = f"ext::{target}_{random.randint(1,9999)}"
                    if tid not in id_map:
                        id_map[tid] = {"id":tid,"text":target,"line":None,"ord":None}
                        units.append(id_map[tid])
                    add_edge(u["id"], tid, 2.0)
    node_list = [{"id":u["id"], "text":u["text"], "line":u.get("line",None)} for u in units]
    edge_list = [(a,b,w) for (a,b),w in edges.items()]
    return node_list, edge_list

# snapshot renderer -> returns PIL Image
def snapshot_graph_image(nodes, edges, energy=0.0, title=None, label_nodes=False):
    G = nx.Graph()
    for n in nodes:
        nid = n["id"] if isinstance(n, dict) else str(n)
        lab = n.get("text", nid) if isinstance(n, dict) else str(n)
        G.add_node(nid, label=lab, line=(n.get("line") if isinstance(n, dict) else None))
    for u,v,w in edges:
        uid = u if isinstance(u,str) else (u.get("id") if isinstance(u, dict) else str(u))
        vid = v if isinstance(v,str) else (v.get("id") if isinstance(v, dict) else str(v))
        if uid!=vid:
            G.add_edge(uid, vid, weight=float(w))
    if len(G.nodes)==0:
        return make_blank_frame()
    n_nodes = max(1, len(G.nodes))
    k = 0.8 / math.sqrt(n_nodes)
    pos = nx.spring_layout(G, seed=42, weight="weight", k=k, iterations=160)
    rng = np.random.RandomState(42)
    for key in list(pos.keys()):
        pos[key] = pos[key] + rng.uniform(-0.02,0.02,size=2)
    # create matplotlib canvas and draw
    fig, ax = plt.subplots(figsize=(W/100, H/100), dpi=100)
    ax.set_facecolor((0.03,0.03,0.05))
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_xlim(-1.1,1.1); ax.set_ylim(-1.1,1.1)
    weights = [d.get("weight",1.0) for _,_,d in G.edges(data=True)]
    maxw = max(weights) if weights else 1.0
    for u,v,d in G.edges(data=True):
        x1,y1 = pos[u]; x2,y2 = pos[v]
        lw = 0.6 + 2.2*(d.get("weight",1.0)/maxw)
        alpha = 0.25 + 0.75*float(energy)
        ax.plot([x1,x2],[y1,y2], lw=lw, color=(0.6,0.8,1.0,alpha))
    deg = dict(G.degree(weight="weight"))
    maxdeg = max(deg.values()) if deg else 1.0
    for n in G.nodes():
        x,y = pos[n]
        dval = deg.get(n, 0.0)
        norm = (dval/maxdeg) if maxdeg>0 else 0.0
        r = 0.02 + 0.06*(1+norm)
        brightness = 0.2 + 0.8*float(energy)
        circ = plt.Circle((x,y), r, color=(brightness*0.2, brightness*0.5, brightness*1.0), ec=(0.1,0.12,0.15), lw=1.0)
        ax.add_patch(circ)
    if title:
        ax.text(-0.98, 0.95, title, color=(0.8,0.9,1.0), fontsize=10)
    # grab as PIL image
    fig.canvas.draw()
    # img = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    # --- FIX for matplotlib >=3.9 ---
    canvas = fig.canvas
    w, h = canvas.get_width_height()
    try:
        buf = canvas.tostring_rgb()  # older versions
    except AttributeError:
        buf = canvas.buffer_rgba()   # newer versions
    img = Image.frombuffer('RGBA' if len(buf) == w*h*4 else 'RGB', (w, h), buf, 'raw')

    if img.mode != "RGB":
        img = img.convert("RGB")

    plt.close(fig)
    if label_nodes:
        # The labeled image will be returned; labels will be drawn later via blending to create fade-in
        draw = ImageDraw.Draw(img)
        # font = get_font(12)
        font = get_font(28)
        for n in G.nodes():
            x,y = pos[n]
            px = int(((x+1)/2.0)*W)
            py = int(((1-y)/2.0)*H)
            lab = G.nodes[n].get("label","")
            draw.text((px,py), str(lab), fill=(255,255,255), font=font, anchor="mm")
        return img
    else:
        return img

# function: blend unlabeled->labeled frames to create label fade-in frames
def labeled_fade_frames(unlabeled_img, labeled_img, steps=ORBIT_LABEL_FADE_STEPS):
    frames=[]
    for i in range(steps):
        alpha = (i+1)/steps
        blended = Image.blend(unlabeled_img, labeled_img, alpha=alpha)
        frames.append(blended)
    return frames

# pulse stanza renderer: returns list of PIL images in order (to be converted to ImageClips)
def render_pulse_stanza_images(stanza_idx, stanza):
    out_images=[]
    lines = stanza.get("lines", [])
    # collect couplets (es,en)
    couplets=[]
    i=0
    while i < len(lines)-1:
        a=lines[i]; b=lines[i+1]
        if a.get("lang","").lower().startswith("es") and b.get("lang","").lower().startswith("en"):
            couplets.append((a,b)); i+=2
        else:
            i+=1
    # compute phon/sem per couplet
    phon_vals=[]; sem_vals=[]; glyph_counts=[]
    for es,en in couplets:
        p = phonetic_similarity(es.get("text",""), en.get("text",""))
        s = semantic_similarity(es.get("text",""), en.get("text",""))
        phon_vals.append(p); sem_vals.append(1.0 - s)
        gcount = len(es.get("glyphs",[])) + len(en.get("glyphs",[]))
        glyph_counts.append(gcount)
    # dynamic evolving trajectory: produce incremental plot images (one per couplet)
    trajectory_imgs=[]
    if len(couplets)>0:
        for idx in range(len(couplets)):
            x = list(range(idx+1))
            phon = phon_vals[:idx+1]
            sem = sem_vals[:idx+1]
            comb = [ph - sc for ph,sc in zip(phon, sem)]
            fig, ax = plt.subplots(figsize=(W/100,H/100))
            ax.set_facecolor((0.02,0.02,0.04))
            times = [i+1 for i in x]
            ax.plot(times, phon, label="phonetic_sim", color=(0.2,0.8,1.0), lw=3)
            ax.plot(times, sem, label="semantic_contrast", color=(1.0,0.5,0.2), lw=3)
            ax.plot(times, comb, label="phon-sem", color=(0.95,0.95,0.6), lw=2, linestyle='--')
            ax.set_xlim(0.5, max(3, times[-1]+0.5))
            ax.set_ylim(-1.0, 1.0)
            ax.legend(loc='upper right', fontsize=10)
            ax.set_title(f"Stanza {stanza.get('id','?')} trajectory (couplet {idx+1})", color='white')
            fig.canvas.draw()
            # traj_img = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
            # --- FIX for matplotlib >=3.9 (compatibility) ---
            canvas = fig.canvas
            w, h = canvas.get_width_height()
            try:
                buf = canvas.tostring_rgb()  # for matplotlib <3.9
            except AttributeError:
                buf = canvas.buffer_rgba()   # for matplotlib ≥3.9
            traj_img = Image.frombuffer('RGBA' if len(buf) == w*h*4 else 'RGB', (w, h), buf, 'raw')

            if traj_img.mode != "RGB":
                traj_img = traj_img.convert("RGB")

            plt.close(fig)
            # center on black background
            bg = make_blank_frame()
            fg = traj_img.resize((int(W*0.8), int(H*0.45)), Image.Resampling.LANCZOS)
            pasted = paste_center(bg, fg, offset=(0,-50))
            trajectory_imgs.append(pasted)
    # now for each couplet, create sequence: ES flash (0.8s), EN flash (0.8s), glyph flash (0.6s), then trajectory snapshot (0.7s)
    for idx, (es,en) in enumerate(couplets):
        # ES frame
        f_es = make_blank_frame()
        draw = ImageDraw.Draw(f_es)
        font = get_font(36)
        # font = get_font(47)
        draw.text((W//2, H//2 - 10), es.get("text",""), font=font, fill=(240,240,240), anchor="mm")
        # EN frame
        f_en = make_blank_frame()
        draw2 = ImageDraw.Draw(f_en)
        draw2.text((W//2, H//2 - 10), en.get("text",""), font=font, fill=(160,210,255), anchor="mm")
        out_images.append( (f_es, COUPLET_FLASH_SECONDS) )
        out_images.append( (f_en, COUPLET_FLASH_SECONDS) )
        # glyph flashes: render glyphs from both lines if any
        glyphs = (es.get("glyphs",[]) or []) + (en.get("glyphs",[]) or [])
        for g in glyphs:
            fg_img = make_blank_frame()
            dd = ImageDraw.Draw(fg_img)
            # attempt to place glyph near target_word: approximate x by char index
            target = g.get("target_word","")
            jitter = g.get("jitter",[12,6])
            # approximate x offset by finding target in either es/en text
            posx = W//2
            posy = H//2
            for txt in (es.get("text",""), en.get("text","")):
                if target and target in txt:
                    idx_char = txt.index(target)
                    pct = idx_char / max(1, len(txt))
                    posx = int(W*0.1 + pct*W*0.8) + random.randint(-jitter[0], jitter[0])
                    posy = H//2 + random.randint(-jitter[1], jitter[1])
                    break
            fontg = get_font(33)
            dd.text((posx,posy), g.get("char", ""), font=fontg, fill=(220,240,200))
            out_images.append( (fg_img, GLYPH_FLASH_SECONDS) )
        # trajectory snapshot for this couplet (if exists)
        if trajectory_imgs:
            traj = trajectory_imgs[idx]
            out_images.append( (traj, max(0.6, COUPLET_FLASH_SECONDS*0.75)) )

    # final pulse stanza consolidated image (short)
    if trajectory_imgs:
        final_traj = trajectory_imgs[-1]
    else:
        final_traj = make_blank_frame()
    out_images.append( (final_traj, FINAL_LABEL_SECONDS) )
    # convert to list of (PIL img, duration)
    return out_images

# helper to convert (PIL,dur) list to moviepy ImageClip list (using numpy arrays)
def images_to_clips(images_with_durations):
    clips=[]
    for img,dur in images_with_durations:
        arr = pil_to_np(img)
        clip = ImageClip(arr).with_duration(float(dur))
        clips.append(clip)
    return clips

def build_video_from_structure(json_path, out_video):
    data = load_json(json_path)
    stanzas = data.get("stanzas", [])
    all_clips=[]
    final_orbit_labeled_paths=[]
    final_pulse_paths=[]
    for si, stanza in enumerate(stanzas):
        mode = stanza.get("mode","orbiting_recursions")
        dur = stanza.get("duration",10.0)
        print(f"Rendering stanza {si+1} mode={mode} duration={dur}s ...")
        out_dir = TMP_DIR / f"stanza_{si+1}"; out_dir.mkdir(parents=True, exist_ok=True)
        if mode.startswith("orbit"):
            nodes, edges = build_orbit_graph_from_stanza(stanza)
            n_lines = max(1, len(stanza.get("lines",[])))
            per_snap = max( (dur - FINAL_LABEL_SECONDS)/n_lines if dur>FINAL_LABEL_SECONDS else dur/(n_lines+1e-6), 0.5)
            # generate snapshots progressively
            active_nodes=[]
            active_edges=[]
            stanza_frames = []
            energies = [0.4 + 0.1*math.sin(i) for i in range(n_lines+1)]
            for li in range(n_lines):
                new_nodes = [n for n in nodes if n.get("line")==li]
                active_nodes.extend(new_nodes)
                active_ids = {n["id"] for n in active_nodes}
                active_edges = [(u,v,w) for (u,v,w) in edges if (u in active_ids and v in active_ids)]
                img_unlabeled = snapshot_graph_image(active_nodes, active_edges, energy=energies[min(li,len(energies)-1)], title=f"Stanza {si+1} • line {li+1}/{n_lines}", label_nodes=False)
                p = out_dir / f"stanza{si+1}_line{li+1}.png"; save_pil_img(img_unlabeled, p)
                # add single frame (duration per_snap)
                stanza_frames.append( (img_unlabeled, per_snap) )
            # final labeled image
            img_all_unlabeled = snapshot_graph_image(nodes, edges, energy=1.0, title=f"Stanza {si+1} — final", label_nodes=False)
            img_all_labeled = snapshot_graph_image(nodes, edges, energy=1.0, title=f"Stanza {si+1} — final", label_nodes=True)
            # produce fade-in frames blending unlabeled->labeled
            fade_frames = labeled_fade_frames(img_all_unlabeled, img_all_labeled, steps=ORBIT_LABEL_FADE_STEPS)
            # save final labeled for recap
            final_path = out_dir / f"stanza{si+1}_final_labeled.png"
            save_pil_img(img_all_labeled, final_path)
            final_orbit_labeled_paths.append(final_path)
            # append fade frames with short durations to stanza_frames
            for fimg in fade_frames:
                stanza_frames.append( (fimg, FINAL_LABEL_SECONDS / len(fade_frames)) )
            # convert to clips and append
            clips = images_to_clips(stanza_frames)
            all_clips.extend(clips)
        elif mode.startswith("pulse"):
            imgs_with_durs = render_pulse_stanza_images(si, stanza)
            # save final traj image path for recap
            if imgs_with_durs:
                last_img, _ = imgs_with_durs[-1]
                final_p = TMP_DIR / f"stanza{si+1}_pulse_final.png"
                save_pil_img(last_img, final_p)
                final_pulse_paths.append(final_p)
            # convert to clips
            clips = images_to_clips(imgs_with_durs)
            all_clips.extend(clips)
        else:
            # fallback black for duration
            all_clips.append(ImageClip(pil_to_np(make_blank_frame())).with_duration(dur))

    # recap
    recap_tiles = []
    for p in final_orbit_labeled_paths + final_pulse_paths:
        if p.exists():
            # create small padded tile
            im = Image.open(p).convert("RGB").resize((W//2 - 20, H//2 - 20), Image.Resampling.LANCZOS)
            bg = make_blank_frame()
            tile = paste_center(bg, im)
            recap_tiles.append(tile)
    if recap_tiles:
        # flash each quickly, then one 4s grid
        quick = []
        for t in recap_tiles:
            quick.append( (t, 0.25) )
        all_clips.extend(images_to_clips(quick))
        # make 2x2 grid
        # tiles = recap_tiles[:4]
        # tiles = recap_tiles
        grid = make_blank_frame()
        for idx, im in enumerate(recap_tiles):
            w2,h2 = im.size
            # x = (idx % 2) * (W//2)
            x = idx * (W//2)
            # y = (idx // 2) * (H//2)
            y = idx * (H//2)
            grid.paste(im, (x, y))
        # all_clips.append(ImageClip(pil_to_np(grid)).with_duration(4.0))
        all_clips.append(ImageClip(pil_to_np(grid)).with_duration(1.3))

    # concatenate all clips
    final_video = concatenate_videoclips(all_clips, method="compose")
    # attach audio master if exists
    if Path(AUDIO_MASTER).exists():
        aud = AudioFileClip(AUDIO_MASTER)
        try:
            aud = aud.subclipped(0, final_video.duration)
        except Exception:
            try:
                aud = aud.subclip(0, final_video.duration)
            except Exception:
                pass
        try:
            final_video = final_video.with_audio(aud)
        except Exception:
            try:
                final_video = final_video.set_audio(aud)
            except Exception:
                pass
    # write out
    final_video.write_videofile(out_video, fps=FPS, codec="libx264", audio_codec="aac")
    print("Saved:", out_video)


In [96]:

if __name__ == "__main__":
    if not Path(JSON_PATH).exists():
        print("Missing JSON, please add poem_structure.json in working dir.")
        sys.exit(1)
    OUT_VIDEO = "MARGENTO_World_Poem_1_Quantum_AHiRA_4.mp4"
    build_video_from_structure(JSON_PATH, OUT_VIDEO)

Rendering stanza 1 mode=orbiting_recursions duration=15s ...
Rendering stanza 2 mode=pulse_flash_spectral duration=25s ...
Rendering stanza 3 mode=orbiting_recursions duration=25s ...
Rendering stanza 4 mode=pulse_flash_spectral duration=35s ...
Rendering stanza 5 mode=orbiting_recursions duration=20s ...
Rendering stanza 6 mode=pulse_flash_spectral duration=63s ...
Rendering stanza 7 mode=orbiting_recursions duration=25s ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


MoviePy - Building video MARGENTO_World_Poem_1_Quantum_AHiRA_4.mp4.
MoviePy - Writing audio in MARGENTO_World_Poem_1_Quantum_AHiRA_4TEMP_MPY_wvf_snd.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


MoviePy - Done.
MoviePy - Writing video MARGENTO_World_Poem_1_Quantum_AHiRA_4.mp4



MoviePy - Done !
MoviePy - video ready MARGENTO_World_Poem_1_Quantum_AHiRA_4.mp4
Saved: MARGENTO_World_Poem_1_Quantum_AHiRA_4.mp4
